
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [ ]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import ribosome, tRNA_charging

from minime.util import dogma
from minime import *
from minime.util import building
from minime.util.building import add_ribosome, add_RNA_polymerase
from minime.util.building import add_complex_stoichiometry_data, add_transcription_reaction
from minime.util.building import add_complex_modification_data
from minime.util.building import add_metabolic_reactions
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [ ]:
import ecolime

In [ ]:
ijo = cobra.test.create_test_model('ecoli')

# Colton TODOs 
- 1) look into all of the free modifications in the original ME and make sure none of thes reactions are blocked
- [x] 2) Frameshifts (b2891)
- [x] 3) selenocysteine
- 4) Non AUG start codons (15%)
- 5) moaD is a modifier protein that never has to be made
- 6) Correct rpL7/12_mod_1:acetyl modification

In [ ]:
from minime.util.mass import compute_RNA_mass

## Begin by loading metabolites and build Metabolic reactions

In [ ]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Suf"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000


In [ ]:
essential_list = [
    'trdox_c',
    'fldox_c',
    'LI_c',
    'cs_c',
    'fldrd_c',
    'trdrd_c',
    'btn_c',
    '3fe4s_c',
    'palmitate_c',
    'C10H8O5_c', 'C9H9O4_c', # for tRNA modifications
    'NiFeCoCN2_c', 'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model, fix how cs is added to

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

In [ ]:
tRNA_mod_dict = get_tRNA_modifications()

In [ ]:
for mod, components in ecolime.tRNA_charging.trna_modification.iteritems():
    tRNA_mod = ModificationData(mod, me)
    tRNA_mod.enzyme = components['machines']
    tRNA_mod.stoichiometry = components['metabolites']
    if 'carriers' in components.keys():
        for carrier, stoich in components['carriers'].items():
            if stoich < 0:
                tRNA_mod.enzyme += [carrier]
            tRNA_mod.stoichiometry[carrier] = stoich

## Build ribosome and RNA Polymerase

### Add translation-related Subreactions

In [ ]:
for subreaction in ribosome.translation_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    subreaction_data.enzyme = ribosome.translation_subreactions[subreaction]['enzyme']

Selenocysteine and start codons

In [ ]:
# TODO contrant type is 'trna efficiency' does this change anything?
# 1 machine + 1 atp + 1 aa + 1 h2o --> 1 machine-amp + 1 h + 1 ppi
# 1 machine-amp + 1 free tRNA --> 1 machine + 1 amp + 1 charged tRNA
subreaction_data = SubreactionData('fmet_addition_at_START', me)
subreaction_data.enzyme = 'InfB_mono' # Start codon loader enzyme
subreaction_data.stoichiometry = {'h_c': 1, '10fthf_c': -1, 'thf_c':1,
                                 'generic_tRNA_START_met__L_c': -1}

subreaction_data = SubreactionData('sec_addition_at_UGA', me)
subreaction_data.enzyme = 'SelA_deca_mod_10:pydx5p' # Selenocysteine loader enzyme
subreaction_data.stoichiometry = {'h_c': 1, 'h2o_c': -1, 'selnp_c':-1, 'pi_c':1,
                                 'generic_tRNA_UGA_cys__L_c': -1}

# create subreaction for each codon
for codon in dogma.codon_table:
    if dogma.codon_table[codon] == '*':
        stop_codon = codon.replace('T','U')
        stop_enzyme = ribosome.translation_stop_dict.get(stop_codon)
        me.add_metabolites([Complex(stop_enzyme)])
        
        subreaction_data = SubreactionData(stop_codon + '_' + stop_enzyme +
                                           '_mediated_termination', me)
        subreaction_data.enzyme = stop_enzyme
        subreaction_data.stoichiometry = {}
    else:
        full_aa = dogma.amino_acids[dogma.codon_table[codon]]
        amino_acid = full_aa.split('_')[0]
        subreaction_data = SubreactionData(amino_acid + '_addition_at_' \
                                           + codon.replace('T','U'), me)
        tRNA = 'generic_tRNA_' + codon.replace('T','U') + '_' + full_aa
        subreaction_data.enzyme = 'generic_Tuf' # Default AA loader enzyme
        subreaction_data.stoichiometry = {full_aa: -1, 'atp_c': -1, 'h2o_c': -1,
                                          'amp_c': 1, 'h_c': 1, 'ppi_c': 1,
                                          tRNA: -1}

In [ ]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)

building.build_reactions_from_genbank(me, gb_filename, TU_df, tRNA_modifications=tRNA_mod_dict, verbose=False)

In [ ]:
building.add_ribosome(me, verbose=False)

add_RNA_polymerase(me)

Make a dummy reactions. Add a dummy protein in as well.

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [ ]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [ ]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
ME_complex_dict = get_complex_to_bnum_dict(rna_components)
add_complex_stoichiometry_data(me, ME_complex_dict)

In [ ]:
modification_dict = get_protein_modification_dict()
add_complex_modification_data(me, modification_dict)

In [ ]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [ ]:
# chaperones
# todo bmocogdp_c mods
# todo FE-S modifications
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp

## TODO NEXT complete Iron-Sulfur cluster modification reactions. Apply rRNA/tRNA fraction constraint 

Build all complex formation reactions

In [ ]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [ ]:
building.add_dummy_reactions(me, "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA", update=False)

In [ ]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

#building.add_and_update_metabolic_reaction(me, "isc_2fe2s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_2fe2s_formation", isc_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "isc_4fe4s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_4fe4s_formation", isc_cplx.id, False, update=True)

In [ ]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [ ]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_complex(generic=False)

In [ ]:
rxn_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    add_metabolic_reactions(me, reaction_data, rxnToModCplxDict, rxn_info, update=True,
                            create_new=True)

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [ ]:
for generic, components in ecoli_k12.generic_dict.items():
    GenericData(generic, me, components).create_reactions()

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [ ]:
apoACP = me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p')
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    if apoACP in reaction.reactants:
        reaction.complex_dilution_list = {apoACP.id}
        reaction.update()

## Add in translocation

In [ ]:
translocPath = pandas.read_csv(join(ecoli_files_dir, "translocation_pathways.txt"),sep='\t')

for index, row in translocPath.iterrows():
    translocRxn = ProteinTranslocationData(row.Reaction_name, me)
    translocRxn.keff = float(row.Keff)
    translocRxn.costs_complexes = row.Complexes.split(' AND ')

In [ ]:
proteins_sa_coeff_inner={}

transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
for index, row in transloc.iterrows():
    me.metabolites.get_by_id(row.Complex).compartment = row.Complex_compartment
    me.metabolites.get_by_id('protein_'+row.Protein.split('(')[0]).compartment = row.Protein_compartment

    if row.translocase_pathway == 's':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        
        ## This is in preparation for membrane constraint
        mass = me.translation_data.get_by_id(row.Protein.split('(')[0]).mass
        if row.Complex in proteins_sa_coeff_inner.keys():
            proteins_sa_coeff_inner[row.Complex]+=mass*1.21/42.*2
        else:
            proteins_sa_coeff_inner[row.Complex]=mass*1.21/42.*2
    elif row.translocase_pathway == 'r':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)       
    elif row.translocase_pathway == 'p':
        me.translocation_pathways.srp_yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)         
    elif row.translocase_pathway =='t':
        me.translocation_pathways.tat_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway=='a':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        me.translocation_pathways.secA_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='l':
        me.translocation_pathways.lol_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='b':
        me.translocation_pathways.bam_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='y':
        me.translocation_pathways.yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)        
    elif row.translocase_pathway != 'n':
        print row.Complex, row.translocase_pathway        

## Make RNA splicing machinery

In [ ]:
#generic_RNase = GenericData("generic_RNase", me,
#                            {ecoli_k12.replace_divalent(i) for i in generic_RNase_list})
#generic_RNase.create_reactions()

rRNA_machinery = ComplexData("rRNA_containing_excision_machinery", me)
#rRNA_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.rRNA_containing}
rRNA_machinery.stoichiometry = {i: 1 for i in ecoli_k12.rRNA_containing}
rRNA_machinery.create_complex_formation()
rRNA_mod = ModificationData("rRNA_containing_excision", me)
rRNA_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
rRNA_mod.enzyme = rRNA_machinery.id

monocistronic_machinery = ComplexData("monocistronic_excision_machinery", me)
#monocistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.monocistronic}
monocistronic_machinery.stoichiometry = {i: 1 for i in ecoli_k12.monocistronic}
monocistronic_machinery.create_complex_formation()
monocistronic_mod = ModificationData("monocistronic_excision", me)
monocistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
monocistronic_mod.enzyme = monocistronic_machinery.id

polycistronic_machinery = ComplexData("polycistronic_excision_machinery", me)
#polycistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.polycistronic_wout_rRNA}
polycistronic_machinery.stoichiometry = {i: 1 for i in ecoli_k12.polycistronic_wout_rRNA}
polycistronic_machinery.create_complex_formation()
polycistronic_mod = ModificationData("polycistronic_excision", me)
polycistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
polycistronic_mod.enzyme = polycistronic_machinery.id

for t in me.transcription_data:
    n_excised = sum(t.excised_bases.values())
    n_cuts = len(t.RNA_products) * 2
    if n_excised == 0 or n_cuts == 0:
        continue
    RNA_types = list(t.RNA_types)
    n_tRNA = RNA_types.count("tRNA")
    
    if "rRNA" in set(RNA_types):
        t.modifications[rRNA_mod.id] = n_cuts
    elif n_tRNA == 1:
        t.modifications[monocistronic_mod.id] = n_cuts
    elif n_tRNA > 1:
        t.modifications[polycistronic_mod.id] = n_cuts
    else:  # only applies to rnpB
        t.modifications[monocistronic_mod.id] = n_cuts

## Attempt to set keffs

In [ ]:
keffs = get_reaction_keffs(me, verbose=True)
for reaction_id, keff in keffs.items():
    me.reactions.get_by_id(reaction_id).keff = keff

## Clean up and update everything

In [ ]:
me.update()
me.prune()

# TODO address these 

make sure they are actually dead ends

In [ ]:
for metabolites in me.metabolites:
    try:
        if len(metabolites.reactions) == 1 and len(ijo.metabolites.get_by_id(metabolites.id).reactions) > 1:
            print metabolites.id, metabolites.reactions
    except:
        print metabolites.id

Figure out why these are necessary

In [ ]:
r = Reaction('DM_preq1_c')
me.add_reaction(r)
r.reaction = 'preq1_c' + ' <-'

r = Reaction('DM_ahcys_c')
me.add_reaction(r)
r.reaction = 'ahcys_c' + ' ->'

r = Reaction('DM_amet_c')
me.add_reaction(r)
r.reaction = 'amet_c' + ' <-'

r = Reaction('DM_dmpp_c')
me.add_reaction(r)
r.reaction = 'dmpp_c' + ' <-'

glycogen_demand = (glycogen_g_per_g)*(1./glycogen_mw)*1000. 

In [ ]:
n_genes = len(me.metabolites.query('RNA_b'))
print("number of genes in the model %d (%.1f%%)" % (n_genes, n_genes * 100. / 1650))

One of these demand metabolites are blocked....

## Solve

In [ ]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [ ]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [ ]:
me.reactions.EX_glc__D_e.lower_bound = -100

In [ ]:
expressions = compile_expressions(me)

In [ ]:
solve_at_growth_rate(me, .1, compiled_expressions=expressions)

In [ ]:
binary_search(me, min_mu=0, max_mu=1.1, mu_accuracy=1e-5,
              compiled_expressions=expressions, debug=False, solver = 'gurobi')